In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from libs import Simulator, GeneralMNRC, SmithPredictor, ModifiedSmithPredictor

In [ ]:
def rpm_per_volt_to_kb(rpm_per_volt):
    rev_per_s_per_V = rpm_per_volt / 60.0
    rad_per_s_per_V = rev_per_s_per_V * 2 * np.pi
    return 1.0 / rad_per_s_per_V


params = {
    "dt": 0.01,
    "theta": 0.1,  # dead time (s)
    "r": 0.05,  # wheel radius (m)
    "m": 2.0,  # robot mass (kg)
    "Jv": 0.125,  # rotational inertia (kg*m^2)
    "kt": 0.025,  # N*m / A
    "kb": rpm_per_volt_to_kb(460.0),  # V/(rad/s) approx, from 460 rpm/V spec
    "ra": 1.2,  # armature resistance (ohm)
    "ngb": 27.0,  # gearbox ratio (motor:wheel)
    "psi": np.deg2rad(np.array([90.0, 90.0, 90.0])),  # wheel traction angle psi_i
    "L": np.array([0.17, 0.17, 0.17]),  # distances Li
    "delta": np.deg2rad(
        np.array([0.0, 120.0, 240.0])
    ),  # wheel angular positions (robot fixed frame)
    "noise_p": 0.01,
    "noise_v": 0.005,
}

In [ ]:
sim = Simulator(params)

f_func1 = lambda y, **kwargs: -sim.M2 @ y
g_func1 = lambda y, **kwargs: sim.M1

Lambda_ref1 = np.array([15.0, 15.0, 15.0])
zeta1 = np.array([1.0, 1.0, 1.0])
tau1 = 1.0 / Lambda_ref1

ctrl1 = GeneralMNRC(f_func1, g_func1, Lambda_ref1, zeta1, tau1, params["dt"])

f_func2 = lambda y, **kwargs: -sim.M3 @ y
g_func2 = lambda y, phi, **kwargs: sim.M3 @ sim.rotation(phi).T @ sim.M4

Lambda_ref2 = np.array([5.0, 5.0, 5.0])
zeta2 = np.array([1.0, 1.0, 1.0])
tau2 = 1.0 / Lambda_ref2

ctrl2 = GeneralMNRC(f_func2, g_func2, Lambda_ref2, zeta2, tau2, params["dt"])

smith2 = SmithPredictor(Lambda_ref2, params["dt"], sim.delay_steps)

f_func3 = lambda y, **kwargs: np.zeros(3)
g_func3 = lambda y, **kwargs: np.eye(3)

Lambda_ref3 = np.array([1.0, 1.0, 1.0])
zeta3 = np.array([1.0, 1.0, 1.0])
tau3 = 1.0 / Lambda_ref3

ctrl3 = GeneralMNRC(f_func3, g_func3, Lambda_ref3, zeta3, tau3, params["dt"])

P1 = np.array([0.5, 0.5, 0.5])  # Not used if ctrl provided
P2 = np.array([2.0, 2.0, 2.0])
mod_smith3 = ModifiedSmithPredictor(
    Lambda_ref2, P1, P2, params["dt"], sim.delay_steps, ctrl=ctrl3
)

#  Codes to reproduce fig 7 (1st level only)

In [ ]:
T = 11.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
omega_r_hist = np.zeros((3, steps))
v_hist = np.zeros((3, steps))

sim = Simulator(params)  # Reset sim

for k in range(steps):
    if k * dt >= 10:
        omega_r = np.array([0, 0, 0])
    elif k * dt >= 7:
        omega_r = np.array([30, 0, -50])
    elif k * dt >= 4:
        omega_r = np.array([30, 0, 40])
    else:
        omega_r = np.array([30, 30, 30])

    v = ctrl1.control(sim.omega, omega_r)
    sim.inner_step(v)
    # No vehicle_step needed for level 1 only

    omega_hist[:, k] = sim.omega
    v_hist[:, k] = v
    omega_r_hist[:, k] = omega_r

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, omega_hist[i],label=rf"$\omega_{i+1}$")
    axs[i].plot(t, omega_r_hist[i],"--",label=rf" reference $\omega_{i+1}$")
    axs[i].set_ylabel(rf"$\omega_{i+1}$ (rad/s)")
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 7: Wheel Speeds")
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, v_hist[i])
    axs[i].set_ylabel(rf"$v_{i+1}$ (V)")
    axs[i].grid(True)
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 7: Voltages")
plt.tight_layout()
plt.show()

# Codes to reproduce fig 9 (1st and 2nd level)

In [ ]:
T = 12.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
vel_hist = np.zeros((3, steps))
r_omega_hist = np.zeros((3, steps))
r_v_hist = np.zeros((3, steps))


sim = Simulator(params)  # Reset sim

for k in range(steps):
    if k * dt >= 9:
        r_v = np.array([0.0, 0.0, 0.0])
    elif k * dt >= 7:
        r_v = np.array([0.0, 0.0, 3.0])
    elif k * dt >= 4:
        r_v = np.array([0, 0.4, 0.0])
    elif k * dt >= 2:
        r_v = np.array([-0.4, 0.0, 0.0])
    else:
        r_v = np.array([0.0, 0.0, 0.0])

    vel_m = sim.vel_history[0]
    predicted_vel = smith2.get_predicted(vel_m)
    phi_m = sim.p[2]  # Approx, since no full update
    r_omega = ctrl2.control(predicted_vel, r_v, phi=phi_m)
    smith2.update(r_v)
    v = ctrl1.control(sim.omega, r_omega)
    sim.inner_step(v)
    sim.vehicle_step()

    omega_hist[:, k] = sim.omega
    vel_hist[:, k] = sim.vel
    r_omega_hist[:, k] = r_omega
    r_v_hist[:,k] = r_v

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels_vel = [
    r"$\dot{x}_c^F$ (m/s)",
    r"$\dot{y}_c^F$ (m/s)",
    r"$\dot{\phi}_c^F$ (rad/s)",
]
for i in range(3):
    axs[i].plot(t, vel_hist[i],label=labels_vel[i])
    axs[i].plot(t, r_v_hist[i],"--",label="reference")
    axs[i].set_ylabel(labels_vel[i])
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 9: Vehicle Speeds")
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, omega_hist[i])
    axs[i].plot(t, r_omega_hist[i], "--")
    axs[i].set_ylabel(rf"$\omega_{i+1}$ (rad/s)")
    axs[i].grid(True)
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 9: Wheel Speeds and Setpoints")
plt.tight_layout()
plt.show()

# Codes to reproduce fig 10 (all levels)

In [ ]:
T = 12.0
dt = params["dt"]
steps = int(T / dt)
t = np.linspace(0, T, steps)
omega_hist = np.zeros((3, steps))
vel_hist = np.zeros((3, steps))
p_hist = np.zeros((3, steps))
v_hist = np.zeros((3, steps))
r_omega_hist = np.zeros((3, steps))
r_v_hist = np.zeros((3, steps))
r_p_hist = np.zeros((3, steps))

w = 2 * np.pi / 8.0
k_phi = 10.0 / 12.0

sim = Simulator(params)  # Reset sim

for k in range(steps):
    r_p = np.array([0.3 * np.sin(w * t[k]), 0.3 * np.cos(w * t[k]), - k_phi * t[k]])
    r_p_hist[:, k] = r_p

    p_m = sim.p_history[0]
    vel_m = sim.vel_history[0]
    phi_m = p_m[2]

    r_v = mod_smith3.control(r_p, p_m)
    mod_smith3.update(r_v)
    r_v_hist[:, k] = r_v

    predicted_vel = smith2.get_predicted(vel_m)

    r_omega = ctrl2.control(predicted_vel, r_v, phi=phi_m)
    r_omega_hist[:, k] = r_omega

    smith2.update(r_v)

    v = ctrl1.control(sim.omega, r_omega)
    v_hist[:, k] = v

    sim.inner_step(v)
    sim.vehicle_step()

    omega_hist[:, k] = sim.omega
    vel_hist[:, k] = sim.vel
    p_hist[:, k] = sim.p

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels_p = [r"$x_c^F$ (m)", r"$y_c^F$ (m)", r"$\phi_c^F$ (rad)"]
for i in range(3):
    axs[i].plot(t, p_hist[i], label='Measured')
    axs[i].plot(t, r_p_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel(labels_p[i])
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 10: Vehicle Position")
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
labels_vel = [r"$\dot{x}_c^F$ (m/s)", r"$\dot{y}_c^F$ (m/s)", r"$\dot{\phi}_c^F$ (rad/s)"]
for i in range(3):
    axs[i].plot(t, vel_hist[i], label='Velocity')
    axs[i].plot(t, r_v_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel(labels_vel[i])
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 10: Vehicle Velocity")
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, omega_hist[i], label=rf"$\omega_{i+1}$")
    axs[i].plot(t, r_omega_hist[i], '--', label='Setpoint')
    axs[i].set_ylabel("Wheel speed (rad/s)")
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 10: Wheel Speeds")
plt.tight_layout()
plt.show()

fig, axs = plt.subplots(3, 1, figsize=(8, 6), sharex=True)
for i in range(3):
    axs[i].plot(t, v_hist[i], label=rf"$v_{i+1}$")
    axs[i].set_ylabel("Motor voltage (V)")
    axs[i].grid(True)
    axs[i].legend()
axs[2].set_xlabel("Time (s)")
fig.suptitle("Fig. 10: Motor Voltages")
plt.tight_layout()
plt.show()